* Communicate with a model using tf-serving
* Use protobuf to build a request

In [1]:
!pip install grpcio==1.42.0 tensorflow-serving-api==2.7.0

In [2]:
!pip install keras-image-helper

In [3]:
import grpc
import tensorflow as tf
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc
from keras_image_helper import create_preprocessor

In [4]:
host = 'localhost:8500' # this is where our tf-serving is running

channel = grpc.insecure_channel(host)
# use this channel to connest to our predictcion service
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [5]:
stub

In [6]:
preprocessor = create_preprocessor('xception', target_size=(299, 299))

In [7]:
url = "http://bit.ly/mlbookcamp-pants"
X = preprocessor.from_url(url)
X

array([[[[-0.11372548, -0.15294117, -0.19999999],
         [-0.11372548, -0.15294117, -0.19999999],
         [-0.10588235, -0.14509803, -0.19215685],
         ...,
         [-0.01960784, -0.01960784, -0.08235294],
         [-0.04313725, -0.04313725, -0.10588235],
         [-0.11372548, -0.11372548, -0.17647058]],

        [[-0.09019607, -0.12941176, -0.17647058],
         [-0.09019607, -0.12941176, -0.17647058],
         [-0.08235294, -0.12156862, -0.16862744],
         ...,
         [-0.01960784, -0.01960784, -0.08235294],
         [-0.04313725, -0.04313725, -0.10588235],
         [-0.10588235, -0.10588235, -0.16862744]],

        [[-0.09803921, -0.1372549 , -0.18431371],
         [-0.09803921, -0.1372549 , -0.18431371],
         [-0.09019607, -0.12941176, -0.17647058],
         ...,
         [-0.01960784, -0.01960784, -0.08235294],
         [-0.03529412, -0.03529412, -0.09803921],
         [-0.09019607, -0.09019607, -0.15294117]],

        ...,

        [[-0.67058825, -0.7019608 , -0

In [8]:
X.shape

(1, 299, 299, 3)

Now we want to send X to our prediction service

In [13]:
def np_to_protobuf(data):
    '''turns np-array to protobuf'''
    return tf.make_tensor_proto(data, shape=data.shape)

In [14]:
# request object to send to tf serving model
pb_request = predict_pb2.PredictRequest() # more robust than json
pb_request.model_spec.name = 'clothing-model'
pb_request.model_spec.signature_name = 'serving_default'
pb_request.inputs['input_13'].CopyFrom(np_to_protobuf(X))

Now the request is prepared, we will execute it now.

In [15]:
pb_response = stub.Predict(pb_request, timeout=20.0)

In [16]:
pb_response

outputs {
  key: "dense_7"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 1
      }
      dim {
        size: 10
      }
    }
    float_val: -1.2789721488952637
    float_val: -4.017117500305176
    float_val: 0.13038446009159088
    float_val: 1.3974616527557373
    float_val: 8.70844554901123
    float_val: -0.6520311832427979
    float_val: -5.029510974884033
    float_val: 2.9917054176330566
    float_val: -0.3883972764015198
    float_val: -2.4756569862365723
  }
}
model_spec {
  name: "clothing-model"
  version {
    value: 1
  }
  signature_name: "serving_default"
}

In [17]:
# extract prediction values
preds = pb_response.outputs['dense_7'].float_val
preds

[-1.2789721488952637, -4.017117500305176, 0.13038446009159088, 1.3974616527557373, 8.70844554901123, -0.6520311832427979, -5.029510974884033, 2.9917054176330566, -0.3883972764015198, -2.4756569862365723]

In [18]:
classes = ['dress', 'hat', 'longsleeve', 'outwear', 'pants', 'shirt', 'shoes', 'shorts', 'skirt', 't-shirt']

In [19]:
dict(zip(classes, preds))

{'dress': -1.2789721488952637,
 'hat': -4.017117500305176,
 'longsleeve': 0.13038446009159088,
 'outwear': 1.3974616527557373,
 'pants': 8.70844554901123,
 'shirt': -0.6520311832427979,
 'shoes': -5.029510974884033,
 'shorts': 2.9917054176330566,
 'skirt': -0.3883972764015198,
 't-shirt': -2.4756569862365723}